In [2]:
import cv2
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
# ビデオデータの読み込み
filename = "C:\\Users\\flow\\Desktop\\bubbledate\\kasupuGray.mp4"
base_path= "C:\\Users\\flow\\Desktop\\bubble_rasing_result\\"
# 動画ファイルのベース名を取得
video_base_name = os.path.splitext(os.path.basename(filename))[0]

# 動画ごとのディレクトリを作成
video_directory = os.path.join(base_path, video_base_name)
os.makedirs(video_directory, exist_ok=True)  # 既に存在している場合は作成しない
# 新しいフォルダのベース名を指定
folder_names = ["no_conditions_image", "no_conditions_contours", "no_conditions_reslut", "no_conditions_graph"]

# 新しいフォルダを作成
for folder_name in folder_names:
    new_folder_path = os.path.join(video_directory, folder_name)
    os.makedirs(new_folder_path, exist_ok=True)

# 動画の読み込み
cap = cv2.VideoCapture(filename=filename)
#動画ファイルの情報を取得する
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
hight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print("総フレーム数: ", total_frame)
print("fps: ", fps)
print("動画の長さ: ", total_frame/fps, "[sec]")

frame_count = 0  # フレームのカウントを初期化

# グラフ用のデータ格納リスト
area_data = []
perimeter_data = []
moment_data = []


while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        # フレーム番号が30の倍数のときだけ処理を行う
        if frame_count % 30 == 0:
            num = frame_count // 30  # 画像ファイルの番号を計算
        # 各処理のpathをvideo_directoryに基づいて生成
            path = os.path.join(video_directory, "no_conditions_image\\")
            contours_path = os.path.join(video_directory, "no_conditions_contours\\")
            result_path = os.path.join(video_directory, "no_conditions_reslut\\")
            
            # 画像を保存
            image_path = path + "picture{:0=3}.bmp".format(num)
            cv2.imwrite(image_path, frame)

            # 画像を読み込み
            color_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            if color_image is None:
                print(f"Error: Failed to load image from {image_path}")
                sys.exit(1)  # スクリプトを終了する

            # グレースケールに変換
            gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

            # 画像の平滑化（ガウシアンフィルタを使用）
            blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 1)

            # Cannyエッジ検出
            edges = cv2.Canny(blurred_image, threshold1=100, threshold2=250)

            # エッジ画像を保存
            edges_path = contours_path + "edges_picture{:0=3}.bmp".format(num)
            cv2.imwrite(edges_path, edges)

            # 輪郭検出
            contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            print(len(contours))
            
            # 各輪郭に対してデータを収集および描画
            for idx, contour in enumerate(contours):
                contour_area = cv2.contourArea(contour)
                contour_perimeter = cv2.arcLength(contour, True)
                
                # 面積、周囲長のデータを追加
                area_data.append(contour_area)
                perimeter_data.append(contour_perimeter)

                # 輪郭検出結果を保存（グリーン色で描画された画像）
                result_image = color_image.copy()
                cv2.drawContours(result_image, [contour], -1, (0, 255, 0), 2)
                
                # 輪郭の中心座標を計算
                M = cv2.moments(contour)
                if M['m00'] != 0:
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                
                # 輪郭番号を描画する位置を中心座標に設定
                text_position = (cx, cy)
                
                # 輪郭番号を描画
                cv2.putText(result_image, str(idx), text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
                # 輪郭検出結果を保存
                result_image_path = os.path.join(result_path, "result_picture{:0=3}.bmp".format(num))
                cv2.imwrite(result_image_path, result_image)
                        
            # 検出された輪郭の数を表示
            num_contours = len(contours)
            print("Number of Contours:", num_contours)


        frame_count += 1
    else:
        break

cap.release()  # カメラリソースの解放
cv2.destroyAllWindows()  # ウィンドウの破棄

総フレーム数:  449.0
fps:  29.97002997002997
動画の長さ:  14.981633333333333 [sec]
315
Number of Contours: 315
290
Number of Contours: 290
307
Number of Contours: 307
304
Number of Contours: 304
315
Number of Contours: 315
307
Number of Contours: 307
313
Number of Contours: 313
307
Number of Contours: 307
306
Number of Contours: 306
313
Number of Contours: 313
305
Number of Contours: 305
311
Number of Contours: 311
311
Number of Contours: 311
301
Number of Contours: 301
309
Number of Contours: 309
